In [1]:
!/usr/local/cuda/bin/nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [2]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [3]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-n_9yugc1
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-n_9yugc1
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit aac710a35f52bb78ab34d2e52517237941399eff
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4287 sha256=4825a60dac8fe70e01bb01115640615ecc7de6863a1b9b228e0b8086b75f7eb4
  Stored in directory: /tmp/pip-ephem-wheel-cache-0hi9ojwi/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built NVCCPlugin


In [4]:
# Commented out IPython magic to ensure Python compatibility.
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [ ]:
# Commented out IPython magic to ensure Python compatibility. 
%%cu 

 #include <cuda.h>
 #include <stdio.h>
 #include <stdlib.h>
 #include <time.h>
 #define m 10
 __global__ void mul_r(int *a, int *b, int *c){
 
 			int tid = threadIdx.x;
 			if (tid < m){
 						c[tid]= a[tid] * b[tid];
 						}
 
 		}
 
 	
 	
 int main(){
 			int  n, c, d, fst[10][10], snd[10][10], t_snd[10][10];
 			int row,col,sum_c, a[10], b[10], ans[10];
 			
  /*
    printf("Enter the number of rows and columns of matrix\n");
    scanf("%d%d", &m, &n);
    */
 n=m;  //square matrix only
    //printf("Enter the elements of first matrix\n");
    //for true randum value of vector
 //srand(time(0));
    for (c = 0; c < m; c++)
    {
    
       for (d = 0; d < n; d++)
 	  {
          //scanf("%d", &first[c][d]);
 		 
 		 fst[c][d]=rand()%10+1;
 		}
 	}
 	printf("display the elements of first matrix\n");	 
 	for (c = 0; c < m; c++) {
       for (d = 0 ; d < n; d++) {
         
 							printf("%d\t", fst[c][d]);
 								}
 								 printf("\n");	 
 							}
      
 	// take next matrix
 	//for true randum value of vector
  //srand(time(0));
    for (c = 0; c < m; c++)
    {
    
       for (d = 0; d < n; d++)
 	  {
          //scanf("%d", &first[c][d]);
 		 
 		 snd[c][d]=rand()%10+1;
 		}
 	}
 	printf("display the elements of second matrix\n");	 
 	for (c = 0; c < m; c++) {
       for (d = 0 ; d < n; d++) {
         
 							printf("%d\t", snd[c][d]);
 								}
 								 printf("\n");	 
 							}
 	// transpose of  second matrix
 	for(c=0; c<m; c++)
         for(d=0; d<n; d++)
         {
             t_snd[d][c] = snd[c][d];
         }
 
     // Displaying the transpose of matrix a
     printf("\nTranspose of second Matrix:\n");
     for (c = 0; c < n; c++) {
       for (d = 0 ; d < m; d++) {
         
 						printf("%d\t", t_snd[c][d]);
 					
 								}
 								 printf("\n");	 
 							}
 					
     // now multiply on cuda
 	int *dev_a, *dev_b,*dev_ans;
 	cudaError_t err=cudaSuccess;
 	// allocate memory on GPU
 	err=cudaMalloc((void**)&dev_a,m * sizeof(int)); 
 	if (err !=cudaSuccess)
 	{ 	printf("failed to  allocate on device \n");
 	printf("error is:\n",cudaGetErrorString(err));
 	exit(EXIT_FAILURE);
 	}
 	//printf("first  ok\n");
 	cudaMalloc((void**)&dev_b,m * sizeof(int)); 
 	cudaMalloc((void**)&dev_ans,m * sizeof(int)); 
 	//printf("first finished ok\n");
 	row=0;
 	col=0;
   cudaEvent_t start, end;
   cudaEventCreate(&start);
   cudaEventCreate(&end);
   cudaEventRecord(start);	  
 
 	for(row=0; row<m; row++){
 		
 	 for (d = 0 ; d < m; d++) {
         
 						a[d]=fst[row][d];
 					
 								}
 							// printf("ok a\n");
 		cudaMemcpy(dev_a,a,m*sizeof(int), cudaMemcpyHostToDevice);	
 	for (col=0; col<m; col++){	
 	 for (d= 0 ; d < m; d++) {
         
 						b[d]=t_snd[col][d];
 						ans[d]=0;
 					
 								}
 							// printf("ok b\n");
 		cudaMemcpy(dev_b,b,m*sizeof(int), cudaMemcpyHostToDevice);	
 		cudaMemcpy(dev_ans,ans,m*sizeof(int), cudaMemcpyHostToDevice);
 	//	printf("calling GPU\n");
 		mul_r<<<1,m>>>(dev_a,dev_b,dev_ans);
 		err=cudaMemcpy(ans,dev_ans,m*sizeof(int), cudaMemcpyDeviceToHost);
 		if (err !=cudaSuccess)
 			{ 	printf("failed to  copy from device \n");
 				exit(EXIT_FAILURE);
 			}
 	//printf("GPU returned\n");						
 	//a=fst[0];
 	sum_c=0;
 	 for (d = 0 ; d < m; d++) {
         
 						//printf("%d\t", ans[d]);
 					sum_c+=ans[d];
 								}
 	snd[row][col]=sum_c;
 	
 	
 	
 	//printf("one element=%d\n",snd[row][col]);
 								// printf("\n");	 
 							
 	}
 	}
 	//
 	cudaEventRecord(end);
   cudaEventSynchronize(end);
   float time = 0;
   cudaEventElapsedTime(&time, start, end);
   printf("execution time=%f\n",time);
 //
 	printf(" Matrix multipliation ans=:\n");
     for (c = 0; c < n; c++) {
       for (d = 0 ; d < m; d++) {
         
 						printf("%d\t", snd[c][d]);
 					
 								}
 								 printf("\n");	 
 							}
 	//
 			return 0;
 				}
 

SyntaxError: invalid syntax (4116449510.py, line 10)